In [1]:
from fastai import *
from fastai.vision import *
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import dlib
import subprocess
import random
import os
bs=8
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
## Declaring path of dataset
path_img = Path('/root/model_training/intern_images')

In [4]:
#function to create list of cv2-loaded images in a given directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),1)
        if img is not None:
            images.append(img)
    return images

#function to convert cv2-loaded image into RGB
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def scale_image(image):
    scale_percent = 25 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

def detectFaceOpenCVDnn(net, image):
    frameHeight = image.shape[0]
    frameWidth = image.shape[1]
    data = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
    
    net.setInput(data)
    detections = net.forward()
    bounding_boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bounding_boxes.append([x1, y1, x2, y2])
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return image, bounding_boxes

In [ ]:
# model to use to locate faces
# haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
conf_threshold = 0.95
DNN = "CAFFE"
if DNN == "CAFFE":
    modelFile = "models/weights.caffemodel"
    configFile = "models/deploy.prototxt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
else:
    modelFile = "opencv_face_detector_uint8.pb"
    configFile = "opencv_face_detector.pbtxt"
    net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

    
for dirname, dirnames, filenames in os.walk('./intern_images'):
    for subdirname in dirnames:
        #print(os.path.join(dirname, subdirname))
        imgs = load_images_from_folder(os.path.join(dirname, subdirname))
        print('loaded images for '+ subdirname)
        for img in imgs:
            # faces = haar_cascade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 4, minSize=(500, 500))
            overlay_image, faces = detectFaceOpenCVDnn(net,img)
            print('Faces found: ', len(faces))
            #for (x,y,w,h) in faces: cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            plt.figure()
            plt.imshow(convertToRGB(overlay_image))

loaded images for sam
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1


/opt/conda/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
loaded images for bilal
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1
Faces found:  1


In [ ]:
img_raw = cv2.imread(path_img+'max/max_001.JPG',1)
type(img_raw)
img_raw.shape
plt.imshow(img_raw)
cv2.imwrite(os.path.join(path , 'waka.jpg'), img)
cv2.waitKey(0)

In [ ]:
## applying transforms
tfms_list = [rotate(degrees=270, p=1),jitter(magnitude=(random.randrange(-3,3)/100), p=0.25), 
            contrast(scale=(0.5, 2.), p=0.5), brightness(change=(0.1, 0.9), p=0.5)]
tfms_list = [rotate(degrees=270, p=1)]
tfms = [tfms_list, tfms_list]

## Loading data 
data = ImageDataBunch.from_folder(path=path_img, train='/', valid_pct=0.07, ds_tfms=tfms, bs=bs, size=(540,720))

## Normalizing data based on ImageNet parameters
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=2)

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
face_detector = dlib.get_frontal_face_detector()
for file_name in data.items:
    img = dlib.load_rgb_image(str(file_name))
    detected_faces = face_detector(img)
    print("Found {} faces in the image file {}".format(len(detected_faces), file_name))

In [ ]:
def get_ex(): return open_image(path_img/'max/max_001.JPG')

def plots_f(rows, cols, width, height, **kwargs):
    [get_ex().apply_tfms(tfms[0], **kwargs).show(ax=ax) for i,ax in enumerate(plt.subplots(
        rows,cols,figsize=(width,height))[1].flatten())]


In [ ]:
## To create a ResNET 50 with pretrained weights
learn = cnn_learner(data, models.resnet50, metrics=accuracy)
print(learn)

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
print('Training results:  ' + str(learn.validate(learn.data.train_dl)))

In [ ]:
print('Validation results: ' + str(learn.validate(learn.data.valid_dl)))

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(4, figsize=(5,5))

In [ ]:
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
learn.lr_find()
learn.recorder.plot()